In [12]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.interpolate import griddata
import regionmask
import glob
import os

In [13]:
path = '/home/rcostell/Observation_Files/CALIPSO_GOCCP/3D_CloudFraction/'

file_pattern = '*.nc'

files = sorted(glob.glob(os.path.join(path, file_pattern)))

f = xr.open_mfdataset(files, combine='by_coords', decode_cf=True)

f['time'] = xr.decode_cf(f)['time']
f = f.resample(time='1M').mean()

lat0 = 60
lat1 = 90
f = f.rename({'latitude':'lat'}).rename({'longitude':'lon'})
f = f.sel(lat = slice(lat0, lat1))
lon_plot = f.lon.values
lat_plot = f.lat.values

#land_mask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(lon_plot, lat_plot)
#f = f.where(land_mask != 0)

f

/home/rcostell/.conda/envs/met2025/lib/python3.10/site-packages/xarray/groupers.py:490: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 498MB
Dimensions:     (time: 96, nv: 2, altitude: 40, lat: 30, lon: 360)
Coordinates:
  * lat         (lat) float32 120B 60.5 61.5 62.5 63.5 ... 86.5 87.5 88.5 89.5
  * lon         (lon) float32 1kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time        (time) datetime64[ns] 768B 2007-01-31 2007-02-28 ... 2014-12-31
Dimensions without coordinates: nv, altitude
Data variables:
    alt_bound   (time, nv, altitude) float32 31kB dask.array<chunksize=(1, 2, 40), meta=np.ndarray>
    alt_mid     (time, altitude) float32 15kB dask.array<chunksize=(1, 40), meta=np.ndarray>
    clcalipso   (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
    clrcalipso  (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
    uncalipso   (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
Attributes:
    Description:               GOCCP_Three-dimensional_Cloud_Fraction_file
    Date:                      
    Version:                   Prog_version
    Author:                    Gregory CESANA, Helene CHEPFER, LMD/IPSL
    Scientific_contact:        helene.chepfer@lmd.polytechnique.fr
    Technical_support:         gregory.cesana@lmd.polytechnique.fr
    Creationdate:              20190201
    Website:                   http://climserv.ipsl.polytechnique.fr/cfmip-ob...
    References:                Chepfer et al, 2010, The GCM Oriented CALIPSO ...
    history:                   Wed Oct  9 18:18:17 2019: ncks -O -4 -L 1 3D_C...
    NCO:                       netCDF Operators version 4.7.9 (Homepage = htt...
    nco_openmp_thread_number:  1

In [14]:
path = '/home/rcostell/Observation_Files/NSIDC/Regridded_Sea_Ice_Concentration/Monthly/Northern_Hemisphere/'

# Define the pattern to match the files
file_pattern = '*_nh_*regridded*.nc'

# Find all files matching the pattern
files = sorted(glob.glob(path + file_pattern))

f0 = xr.open_mfdataset(files, combine='nested', concat_dim='tdim')
f0 = f0.rename({'tdim': 'time'})
f0 = f0.interp(lat=lat_plot, lon=lon_plot)

ind = []
valid_times = []

tolerance = np.timedelta64(1, 's')

for i, d in enumerate(f.time.values):
    if not np.isnan(d):
        valid_times.append(d)
        tmp = np.where(np.abs(f0.time.values - d) <= tolerance)[0]
        if len(tmp) > 0:
            ind.append(tmp[0])

valid_times = np.array(valid_times)
ind = np.array(ind)

valid_times = valid_times[:len(ind)]

f0 = f0.isel(time = ind).cdr_seaice_conc_gridded

f = f.sel(time=valid_times)

f0['time'] = f.time
f = f.assign(seaice_conc=f0)
f

/tmp/ipykernel_455549/2766318870.py:10: UserWarning: rename 'tdim' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  f0 = f0.rename({'tdim': 'time'})


<xarray.Dataset> Size: 506MB
Dimensions:      (time: 96, nv: 2, altitude: 40, lat: 30, lon: 360)
Coordinates:
  * lat          (lat) float32 120B 60.5 61.5 62.5 63.5 ... 86.5 87.5 88.5 89.5
  * lon          (lon) float32 1kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time         (time) datetime64[ns] 768B 2007-01-31 2007-02-28 ... 2014-12-31
Dimensions without coordinates: nv, altitude
Data variables:
    alt_bound    (time, nv, altitude) float32 31kB dask.array<chunksize=(1, 2, 40), meta=np.ndarray>
    alt_mid      (time, altitude) float32 15kB dask.array<chunksize=(1, 40), meta=np.ndarray>
    clcalipso    (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
    clrcalipso   (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
    uncalipso    (time, altitude, lat, lon) float32 166MB dask.array<chunksize=(1, 30, 30, 266), meta=np.ndarray>
    seaice_conc  (lat, lon, time) float64 8MB dask.array<chunksize=(30, 360, 96), meta=np.ndarray>
Attributes:
    Description:               GOCCP_Three-dimensional_Cloud_Fraction_file
    Date:                      
    Version:                   Prog_version
    Author:                    Gregory CESANA, Helene CHEPFER, LMD/IPSL
    Scientific_contact:        helene.chepfer@lmd.polytechnique.fr
    Technical_support:         gregory.cesana@lmd.polytechnique.fr
    Creationdate:              20190201
    Website:                   http://climserv.ipsl.polytechnique.fr/cfmip-ob...
    References:                Chepfer et al, 2010, The GCM Oriented CALIPSO ...
    history:                   Wed Oct  9 18:18:17 2019: ncks -O -4 -L 1 3D_C...
    NCO:                       netCDF Operators version 4.7.9 (Homepage = htt...
    nco_openmp_thread_number:  1

In [15]:
f = f.sel(lat=slice(lat0, lat1)).sel(lon=slice(-15, 55))
ice_achv = f.seaice_conc.values
cl_achv = f.clcalipso.values
lat = f.lat.values
lon = f.lon.values
time = f.time.values
print(f.isel(time = 0).alt_mid.values)
altitude = f.isel(time = 0).alt_mid.values
nlat = len(lat)
nlon = len(lon)
ntime = len(time)
naltitude = len(altitude)

[ 0.24        0.71999997  1.2         1.6800001   2.16        2.64
  3.12        3.6         4.08        4.5600004   5.04        5.5200005
  6.          6.4799995   6.96        7.4399996   7.92        8.4
  8.88        9.360001    9.84       10.32       10.8        11.280001
 11.76       12.24       12.719999   13.2        13.68       14.16
 14.639999   15.12       15.6        16.08       16.56       17.04
 17.52       18.         18.48       18.96      ]


In [16]:
def Binning_distance_from_ice_edge(ice_achv, var_achv, ntime, nlat, nlon, hem, outputcount = False, model = False):
    for j in range(ntime):
        if model:
            ice = ice_achv[j, :, :]
        else:
            ice = ice_achv[:, :, j]
        var = var_achv[j]
        ndim = var.ndim

        for i in range(nlon):
            tmp = ice[:,i]
            if ndim==3:
                var_tmp = var[:,:,i]
                nh = var_tmp.shape[0]
            else:
                var_tmp = var[:,i]
            
            if hem == 'nh':
                ind1 = np.where(tmp <= 0.5)[0]
                ind2 = np.where(tmp > 0.5)[0]
            else:
                ind1 = np.where(tmp > 0.5)[0]
                ind2 = np.where(tmp <= 0.5)[0]                

            n1 = ind1.size
            n2 = ind2.size

            if i == 0:
                tmp0 = np.full((2*nlat), -999.)
                tmp0[nlat - n1: nlat] = tmp[ind1]
                tmp0[nlat: nlat + n2] = tmp[ind2]

                count0 = np.full((2*nlat), 1.)
                
                if ndim==3:
                    var_tmp0 = np.full((nh, 2*nlat), -999.)
                    var_tmp0[:,nlat - n1: nlat] = var_tmp[:,ind1]
                    var_tmp0[:,nlat: nlat + n2] = var_tmp[:,ind2]
                else:
                    var_tmp0 = np.full((2*nlat), -999.)
                    var_tmp0[nlat - n1: nlat] = var_tmp[ind1]
                    var_tmp0[nlat: nlat + n2] = var_tmp[ind2]
            else:
                tmp1 = np.full((2*nlat), -999.)
                tmp1[nlat - n1: nlat] = tmp[ind1]
                tmp1[nlat: nlat + n2] = tmp[ind2]

                count1 = np.full((2*nlat), 1.)
                
                if ndim==3:
                    var_tmp1 = np.full((nh, 2*nlat), -999.)
                    var_tmp1[:,nlat - n1: nlat] = var_tmp[:,ind1]
                    var_tmp1[:,nlat: nlat + n2] = var_tmp[:,ind2]
                else:
                    var_tmp1 = np.full((2*nlat), -999.)
                    var_tmp1[nlat - n1: nlat] = var_tmp[ind1]
                    var_tmp1[nlat: nlat + n2] = var_tmp[ind2]

                tmp0 = np.dstack((tmp0, tmp1))
                count0 = np.dstack((count0, count1))
                var_tmp0 = np.dstack((var_tmp0, var_tmp1))

        var_tmp0[var_tmp0 == -999.] = np.nan
        tmp0[tmp0 == -999.] = np.nan
        
        if ndim == 2:
            count0[np.isnan(var_tmp0)] = 0.
        
        count00 = nlon*np.nanmean(count0[0], axis = 1)
        tmp00 = np.nanmean(tmp0[0], axis = 1)
        if ndim == 3:
            var_tmp00 = np.nanmean(var_tmp0, axis = 2)
        else:
            var_tmp00 = np.nanmean(var_tmp0[0], axis = 1)

        if j == 0:
            count_out = count00
            var_out = var_tmp00
            ice_out = tmp00
        else:
            count_out = np.dstack((count_out, count00))        
            var_out = np.dstack((var_out, var_tmp00))
            ice_out = np.dstack((ice_out, tmp00))

    count_out = count_out[0]
    ice_out = ice_out[0]
    if ndim != 3:
        var_out = var_out[0]
    
    if ndim == 3:
        return var_out, ice_out
    else:
        if outputcount:
            return var_out, ice_out, count_out    
        else:
            return var_out, ice_out

In [17]:
print("Start extracting")
print(cl_achv.shape)

cl_out, ice_out = Binning_distance_from_ice_edge(ice_achv, cl_achv, ntime, nlat, nlon, 'nh')

ds_tmp = xr.DataArray(cl_out, coords=[altitude, np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["altitude", "distance","time"]).rename('clcalipso')
ds_tmp1 = xr.DataArray(ice_out, coords=[np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["distance","time"]).rename('seaice_conc')

ds_out = xr.merge([ds_tmp, ds_tmp1])

#ds_out.to_netcdf("/home/rcostell/Observation_Files/Binned/Binned_Observed_3D_CF.nc")
print("Observed file made")

Start extracting
(96, 40, 30, 70)


/tmp/ipykernel_455549/949459858.py:70: RuntimeWarning: Mean of empty slice
  tmp00 = np.nanmean(tmp0[0], axis = 1)
/tmp/ipykernel_455549/949459858.py:72: RuntimeWarning: Mean of empty slice
  var_tmp00 = np.nanmean(var_tmp0, axis = 2)


Observed file made
